### Objective

The understanding of Shake-Shake Regularization and the implementation of Layer using `tf.keras`.

### 연구의 모티브

딥러닝의 Overfitting 문제를 해결하는 수단을 찾아보자! <br>
$\rightarrow$ 가장 쉬운 방식은 Data Augmentation, 데이터에 노이즈를 잔뜩 주자 <br>
$\rightarrow$ 그래봤자 Data Augmentation은 모델 내부의 표현(Intermediate representation)에는 큰 변화를 주지 못한다.<br> 
$\rightarrow$ 어떻게 모델 내부 표현에 노이즈를 줄까?<br>
$\rightarrow$ 요즘 유행하는 **Multi-Branch**의 구조를 한번 이용해볼까!

### 최근 Mutli-Branch의 대표적인 모델인 Res-Next를 살펴보자!

<Resnet과 ResNext의 차이에 대한 그림>

![](https://miro.medium.com/max/1468/1*LOoc11tkDoqv0pC6OH7mwA.png)


같은 Residual Block 내에서 ResNext는 그룹 단위로 쪼개고(split), 각 그룹 별로 변환하고 (Affine Transform), 각 그룹의 결과를 모아줌으로써 처리한다. <br>
그렇게 하면, 연산량이나 파라미터가 크게 오르지 않을까? $\rightarrow$ 그렇지 않다!

![Imgur](https://i.imgur.com/RgV2KZS.png)

사실 BottleNeck으로 줄였다가(Squeeze), Group Convolution을 적용한 거랑 동일한 구조이다. 그래서 파라미터의 크기는 아래와 같다!

#### ResNet에서의 Param 수 : 

In [27]:
256 * 1 * 1* 64 + 64 * 3 * 3 * 64 + 64 * 1 * 1* 256

69632

#### Res-Next에서의 param 수 : 

In [28]:
32* (256 * 1 * 1* 4 + 4 * 3 * 3 * 4 + 4 * 1* 1* 256)

70144

위와 같이 레이어를 구성하면, 파라미터 수도 거의 비슷하고, 연산량도 거의 비슷하게 유지된다!. 그런데 성능은 크게 향상된다!

![Imgur](https://i.imgur.com/Fyh8EYm.png)

보면 복잡도를 2배씩 올린 모델의 성능 향상보다, Cardiality, 즉 Group으로 나누어 처리하게한 ResNext가 성능향상이 더 되었다!

연구의 모티브는 바로 이 Multi-Branch 아이디어를 활용하여, **"Regularization 효과까지 한번 만들어보겠다"**


### 사전 연구

#### (1) Fractal-Net

이런 아이디어로 처음 나온 것은 Residual Network 없이 충분히 깊게 쌓아보자 라는 아이디어로 만들어진 Fractal-Net이 있다.

![Imgur](https://i.imgur.com/pZbx4Sj.png)

이런 식으로 매 학습때마다 Feed Forward하는 경로를 랜덤하게 결정!(50%확률로 Local 방식으로 path Sampling하고 50% 확률로 Global 방식으로 Path Sampling)

이 사람은 Multi-Branch의 경로를 랜덤하게 해줌으로써 모델에 Regularization 효과를 제공하는 방식 (어떻게 보면, Dropout의 일종, 뉴런을 켰다 껐다 하는 대신 작은 네트워크를 켰다 껐다 하는 방식)

#### (2) Shakeout 

껐다 켰다 하는 Dropout 대신, 뉴런의 출력을 변경하는 네트워크 (Paper를 구체적으로 살펴보지 못해서 이해하지 못했습니다)

![Imgur](https://i.imgur.com/sRED6I9.png)

---

## Paper Overview

Shake-Shake Network이 어떤 식으로 동작하는지를 설명하는 그림입니다.

![Imgur](https://i.imgur.com/3onSLc5.png)

위의 연산을 이해하기 위해서 먼저 뿌리가 되는 Res-Net부터 살펴보도록 하겠습니다. Residual Network라면 아래와 같이 구성됩니다.
![Imgur](https://i.imgur.com/FpBLEvg.png)

이러한 Res Net은 수식으로 보면 아래와 같은 수식입니다.

$$
x_{i+1} = x_i + \mathcal{F}(x_i,\mathcal{W}_i) 
$$

Shake Shake Regularization에서는 두 개의 Residual Block을 병렬로 두게 됩니다.

![Imgur](https://i.imgur.com/tNwYQWK.png)

즉 위의 연산을 수식으로 나타내면 아래와 같습니다. 

$$
x_{i+1} = x_i + \alpha_i \mathcal{F}(x_i,\mathcal{W}_i^{(1)})  + (1- \alpha_i) \mathcal{F}(x_i,\mathcal{W}_i^{(2)})
$$


이 때의 $\alpha$는 random value로, 0과 1사이의 균등분포에서 무작위로 추출한 값이 됩니다. 즉 왼쪽과 오른쪽의 브랜치에 대한 피처맵 가중치는 매번 바뀌는 값이 됩니다. 

Shake-Shake 네트워크에서 독특한 점은 바로 아래, BackPropagation할 때에 있습니다.

![Imgur](https://i.imgur.com/XDfbzJf.png)

Feed Forward 때와 달리, Backward에서 랜덤 값을 다른 것으로 줍니다. <br>
즉 Gradient의 움직임에 노이즈를 주는 아이디어 입니다. 저자들은 이 아이디어에 대해 아래와 같이 설명합니다. 


> As shown in Figure 1, all scaling coefficients are overwritten with new random numbers before each
forward pass. **The key to making this work is to repeat this coefficient update operation before each
backward pass.** This results in a stochastic blend of forward and backward flows during training.
Related to this idea are the works of An (1996) and Neelakantan et al. (2015). These authors showed
that adding noise to the gradient during training helps training and generalization of complicated
neural networks. Shake-Shake regularization can be seen as an extension of this concept where
gradient noise is replaced by a form of gradient augmentation.



위와 같은 방식으로 모델을 학습시켰을 때, CIFAR-10에서의 Error rate는 아래와 같습니다.

![Imgur](https://i.imgur.com/xRnoX5g.png)


[paper with code - Leader Board of CIFAR-10](https://paperswithcode.com/sota/image-classification-on-cifar-10)에서 살펴보았을 때에도, ShakeShake 아이디어를 접목한 모델들이 상위권에 기재되어 있습니다.

![Imgur](https://i.imgur.com/gZFflr4.png)

## Implementation

Keras로 간결하게, 그리고 매우 잘 구현된 코드가 있습니다.<br>
*(caution : 가독성을 위해, 약간의 refactoring을 거쳤습니다.)* <br>
[keras-shake-shake layers](https://github.com/jonnedtc/Shake-Shake-Keras/blob/master/layers.py)

In [2]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer

class ShakeShake(Layer):
    """ Shake-Shake-Image Layer """
    def __init__(self, verbose=False, **kwargs):
        self.verbose = verbose
        super().__init__(**kwargs)
    
    def call(self, inputs):
        assert isinstance(inputs, list)
        left, right = inputs # unpack left and right
        
        batch_size = K.shape(left)[0]
        alpha = K.random_uniform((batch_size, 1, 1, 1))
        beta = K.random_uniform((batch_size, 1, 1, 1))

        # Shake-Shake-Image
        def shake():
            forward = alpha * left + (1 - alpha) * right 
            backward = beta * left + (1 - beta) * right
            return backward + K.stop_gradient(forward - backward)
        
        # even during testing phase
        def even():
            return 0.5 * left + 0.5 * right
        
        if self.verbose:
            # Check 되면, alpha & beta값 보임
            op1 = K.print_tensor(alpha, "alpha : ")
            op2 = K.print_tensor(beta, "beta : ")        
            with tf.control_dependencies([op1]):
                with tf.control_dependencies([op2]):
                    return K.in_train_phase(shake, even)
        else:
            return K.in_train_phase(shake, even)

Shake-Shake은 두가지 Input을 받습니다. left와 right는 각각의 branch를 지칭합니다. 위 모델이 잘 돌아가는지를 파악하기 위해서 아래와 같은 테스트 코드를 작성해보도록 하겠습니다.

In [29]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np

In [35]:
K.clear_session()

inputs = Input((1,))

left_branch_layer = Dense(1, use_bias=False)
right_branch_layer = Dense(1, use_bias=False)

left  = left_branch_layer(inputs)
right = right_branch_layer(inputs)
outputs = ShakeShake(verbose=True)([left, right])

model = Model(inputs, outputs, name='shake-shake')

모델의 동작을 보다 간단하게 확인하기 위해, 왼쪽 브랜치의 가중치를 1, 오른쪽 브랜치의 가중치를 -1로 두겠습니다.

In [36]:
left_branch_layer.set_weights(np.array([[[1]]]))
right_branch_layer.set_weights(np.array([[[-1]]]))

학습환경에서의 FeedForward는 alpha값에 따라 아래와 같이 변경됩니다.

In [42]:
K.get_session().run(outputs, feed_dict={inputs:np.array([[1]]),
                                       K.learning_phase(): True})

alpha :  [[[[0.570445538]]]]
beta :  [[[[0.723735452]]]]


array([[[[0.14089108]]]], dtype=float32)

추론 환경(Test 환경)에서는 EVEN, 왼쪽과 오른쪽 모두 동일한 가중치를 주게 되므로, 0이 됩니다.

In [38]:
K.get_session().run(outputs, feed_dict={inputs:np.array([[1]]),
                                       K.learning_phase(): False})

alpha :  [[[[0.208138108]]]]
beta :  [[[[0.836968064]]]]


array([[0.]], dtype=float32)

학습환경에서의 Backward는 Backward값에 따라 변경됩니다.

In [43]:
grad = tf.gradients(outputs, inputs)

K.get_session().run(grad, feed_dict={inputs:np.array([[1]]),
                                     K.learning_phase(): True})

alpha :  [[[[0.869900942]]]]
beta :  [[[[0.399321795]]]]


[array([[-0.20135641]], dtype=float32)]

위의 움직임을 통해 `ShakeShake`코드가 정상적으로 돌아가는 것을 확인할 수 있습니다.

[Optional (1)] ShakeEven은 아래와 같이 작성하면 됩니다.

In [ ]:
class ShakeEven(Layer):
    """ Shake-Even-Image Layer """
    def call(self, inputs):
        assert isinstance(inputs, list)
        left, right = inputs # unpack x1 and x2
        
        batch_size = K.shape(left)[0]
        alpha = K.random_uniform((batch_size, 1, 1, 1))

        # Shake-Even-Image
        def shake_shake():
            forward = alpha * left + (1 - alpha) * right 
            backward = 0.5 * left + 0.5 * right
            return backward + K.stop_gradient(forward - backward)
        
        # even during testing phase
        def even():
            return 0.5 * left + 0.5 * right
        
        return K.in_train_phase(shake, even)    

[Optional (2)] 텐서플로우에서는 아래와 같은 방식으로 Override해서 구현해야 합니다. <br>

reference : [Tensorflow: How to replace or modify gradient](https://stackoverflow.com/questions/43839431/tensorflow-how-to-replace-or-modify-gradient)

![Imgur](https://i.imgur.com/smT72Jb.png)

#### 실제 사용 예시는 아래와 같습니다.

In [ ]:
from tensorflow.keras.layers import ReLU, Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate, Input
from tensorflow.keras.models import load_model, Model

In [ ]:
def create_residual_branch(x, filters, stride):
    """ Regular Branch of a Residual network: ReLU -> Conv2D -> BN repeated twice """
    x = ReLU()(x)
    x = Conv2D(filters, 3, strides=stride, padding='same',
               kernel_initializer='he_normal', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters, 3, strides=1, padding='same',
               kernel_initializer='he_normal', use_bias=False)(x)
    x = BatchNormalization()(x)
    return x


def create_residual_shortcut(x, filters, stride):
    """ Shortcut Branch used when downsampling from Shake-Shake regularization """
    x = ReLU()(x)
    x1 = x[:,0:-1:stride, 0:-1:stride]
    x1 = Conv2D(filters // 2, 1, strides=1, padding='valid',
                kernel_initializer='he_normal', use_bias=False)(x1)
    x2 = x[:,1::stride, 1::stride]
    x2 = Conv2D(filters // 2, 1, strides=1, padding='valid',
                kernel_initializer='he_normal', use_bias=False)(x2)
    x = Concatenate()([x1, x2])
    x = BatchNormalization()(x)
    return x


def create_residual_block(x, filters, stride=1):
    """ Residual Block with Shake-Shake regularization and shortcut """
    x1 = create_residual_branch(x, filters, stride)
    x2 = create_residual_branch(x, filters, stride)
    if stride > 1: 
        x = create_residual_shortcut(x, filters, stride)
    return x + ShakeShake()([x1, x2])

----

## reference

* **논문** : 
    1. [Shake-Shake Regularization](https://arxiv.org/pdf/1705.07485.pdf)
    2. [Aggregated Residual Transformations for Deep Neural Networks](https://arxiv.org/pdf/1611.05431.pdf)
    3. [FractalNet: ULTRA-DEEP NEURAL NETWORKS WITHOUT RESIDUALS](https://arxiv.org/pdf/1605.07648.pdf)
    4. [Shakeout: A New Regularized Deep
Neural Network Training Scheme](https://pdfs.semanticscholar.org/310e/c7796eeca484d734399d9979e8f74d7d8ed2.pdf)

* **블로그** : 
    1. [SUALAB Blog - Shake-Shake Regularization](http://research.sualab.com/practice/review/2018/06/28/shake-shake-regularization-review.html)
    2. [CIFAR-10 정복하기 3: Shake-Shake](https://dnddnjs.github.io/cifar10/2018/10/25/shake_shake/)
    3. [Shake-Shake Regularization with Interactive Code](https://medium.com/@SeoJaeDuk/shake-shake-regularization-with-interactive-code-manual-back-prop-with-tf-20505cb21a9e)

* **깃헙** : 
    1. [shake-shake-keras/](https://github.com/jonnedtc/Shake-Shake-Keras/blob/master/layers.py)

* **스택오버플로우** : 
    1. : [tensorflow-how-to-replace-or-modify-gradient](https://stackoverflow.com/questions/43839431/tensorflow-how-to-replace-or-modify-gradient)